<a href="https://colab.research.google.com/github/ajayjai30/SUBJECT_EXPERT_CHATBOT_FOR_STUDETNS_AND_RESEARCHERS/blob/main/PROJECT_SUBJECT_EXPERT_CHATBOT_PHASE_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Section-1 Libraries and Dependencies Installation and setting up of libraries

Dependencies installation section

In [ ]:
#dependencies installation
!pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf
!sudo apt-get -y -qq install poppler-utils
!pip install langchain
!pip install langchain-community
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.0
    Uninstalling tomlkit-0.13.0:
      Successfully uninstalled tomlkit-0.13.0




```
!pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf
!sudo apt-get -y -qq install poppler-utils libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!sudo apt -y -qq install tesseract-ocr libtesseract-dev# for embedding of text inside images
!pip install langchain
!pip install langchain-community
!pip install gradio
```



Libraries Setting

In [ ]:
#importing libraries
import os#for file handling purposes
from google.colab import userdata#used for storing public data when processing Here api will be stone and also for accesing secret access key.
import google.generativeai as genai#library for using LLM model with all features given by google
import urllib
from pathlib import Path as p
from pprint import pprint
import textwrap#For adjusting line breaks and adding indentation inside markdown.
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from IPython.display import display,Markdown#for displaying information as output
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

***Function for editing the response style and also using text to speech methods for additional functionality***


Overall Functionality:

The to_markdown function takes plain text as input, converts bullet points to Markdown-style lists, indents all lines to create a blockquote, and returns the result as a Markdown object. This is useful for converting text into a format suitable for display on platforms that support Markdown, such as GitHub, Reddit, or various blogging platforms.

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#API key setup and verification of models availability

Section-1
setting up the configuration of Model from api key

In [ ]:
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

verification of models for synchronizing with the environment

In [ ]:
for model in genai.list_models():
  if 'generateContent' in model.supported_generation_methods:
    print(model.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash
models/gemini-1.5-flash-001-tuning


#Environment creation and Initialization of model

Setting up the environment for the model

In [ ]:
import os
os.environ['GOOGLE_API_KEY']=userdata.get("GOOGLE_API_KEY")

Initiating the model inside the environment

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

#Extracting text from pdf files located in drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
os.listdir(Main_path)[i]

In [ ]:
i

1

In [ ]:
import os
all_pages=[]
Main_path='/content/drive/MyDrive/contentconvertedpdf'

for i in range(len(os.listdir(Main_path))):
     j=os.path.join(Main_path,os.listdir(Main_path)[i])
     pdf_loader = PyPDFLoader(j)
     pages = pdf_loader.load_and_split()
     all_pages.extend(pages)

#print(all_pages[5].page_content)  # Can Be used as a verification for Printing the content of the first page of the first PDF

#workings on problem

In [ ]:
!pip install pdf2image
!pip install pytesseract==0.3.10

In [ ]:
from pdf2image import convert_from_path
from pytesseract import image_to_string as s
from PIL import Image
import os

In [ ]:
def convert_to_img_from_pdf(pdf_file_path):
  return convert_from_path(pdf_file_path)

In [ ]:
def extract_text_from_image(file):
  return s(file)

In [ ]:
all_pages=[]
Main_path='/content/drive/MyDrive/contentconvertedpdf'
for i in range(len(os.listdir(Main_path))):
  j=os.path.join(Main_path,os.listdir(Main_path)[i])
  if j.endswith('.pdf'):
    images=convert_to_img_from_pdf(j)
    for img in images:
      all_pages.append(extract_text_from_image(img))
  else:
    continue

In [ ]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.3 MB/s eta 0:00:00


In [ ]:
import os
from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    text = extract_text(pdf_path)
    return text

all_pages = []
Main_path = '/content/drive/MyDrive/ContentforModelFluidMechanics'

for file in os.listdir(Main_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(Main_path, file)
        text = extract_text_from_pdf(pdf_path)
        all_pages.append(text)

print(all_pages)


In [ ]:
import os
from multiprocessing import Pool
import pdf2image
import pytesseract

def process_pdf(pdf_path):
    images = pdf2image.convert_from_path(pdf_path)
    text_data = []
    for img in images:
        text = pytesseract.image_to_string(img)
        text_data.append(text)
    return text_data

def main():
    all_pages = []
    Main_path = '/content/drive/MyDrive/contentconvertedpdf'

    with Pool(processes=4) as pool:  # Adjust number of processes as needed
        all_pages = pool.map(process_pdf, [os.path.join(Main_path, f) for f in os.listdir(Main_path) if f.endswith('.pdf')])

    # Flatten the list of lists
    all_pages = [page for sublist in all_pages for page in sublist]

    print(all_pages)

if __name__ == '__main__':
    main()


In [ ]:
import os
import PyPDF2
import pytesseract
import pdfplumber
from PIL import Image

def extract1_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            all_text = []
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                all_text.append(page.extract_text())

                try:
                    with pdfplumber.open(file_path) as pdf:
                        page_obj = pdf.pages[page_num]
                        images = page_obj.extract_images()
                        for image in images:
                            image_data = image["data"]
                            img = Image.frombytes(mode=image["colorspace"], size=image["size"], data=image_data)
                            try:
                                text = pytesseract.image_to_string(img)
                                all_text.append(text)
                            except Exception as e:
                                print(f"Error processing image: {e}")
                except Exception as e:
                    print(f"Error processing PDF images: {e}")

            return all_text
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return []

# rest of the code remaining the same


In [ ]:
!pip install pymupdf==1.22.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 78.0 MB/s eta 0:00:00


In [ ]:
import os
import fitz
import pytesseract
from PIL import Image

def extract_text_from_pdf(file_path):
    try:
        doc = fitz.open(file_path)
        all_text = []
        for page_num in range(len(doc)):
            page = doc[page_num]
            all_text.append(page.get_text())

            # Extract images
            for img in page.get_images():
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                img = Image.frombytes(mode=base_image["colorspace"], size=base_image["size"], data=image_bytes)
                try:
                    text = pytesseract.image_to_string(img)
                    all_text.append(text)
                except Exception as e:
                    print(f"Error processing image: {e}")

        doc.close()
        return all_text
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return []

# ... rest of the code remains the same


In [ ]:
import os
import fitz
import pytesseract
from PIL import Image
import numpy as np

def extract_text_from_pdf(file_path):
    try:
        doc = fitz.open(file_path)
        all_text = []
        for page_num in range(len(doc)):
            page = doc[page_num]
            all_text.append(page.get_text())

            # Extract images
            for img in page.get_images():
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                image_info = base_image["image_info"]

                try:
                    img = Image.frombytes(mode=image_info["cs"], size=(image_info["w"], image_info["h"]), data=image_bytes)
                except Exception as e:
                    try:
                        img = Image.fromarray(np.array(image_bytes).reshape((image_info["h"], image_info["w"], 3)), mode='RGB')
                    except Exception as e:
                        print(f"Error processing image: {e}")
                        continue

                try:
                    text = pytesseract.image_to_string(img)
                    all_text.append(text)
                except Exception as e:
                    print(f"Error processing image text: {e}")

        doc.close()
        return all_text
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return []

def extract_text_from_directory(directory):
    all_text = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.pdf'):
                file_path = os.path.join(root, file)
                page_text = extract_text_from_pdf(file_path)
                all_text.extend(page_text)
    return all_text

# Example usage
Main_path = '/content/drive/MyDrive/ContentforModelFluidMechanics'
all_text = extract_text_from_directory(Main_path)
print(all_text)


Error processing file /content/drive/MyDrive/ContentforModelFluidMechanics/2500solvedproblemsinfluidmechanicshydraulics.pdf: 'image_info'
[]


**Trying to use lamma-parser**

#Building pipeline for rag

Here the flow of pipeline is shown below


**pdf==> text generated as chunks ==>chunks converted into embeddings==> uploaded into chroma data base**

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=10)
content="n\n".join(str(p.page_content) for p in all_pages)
text_chunks=text_splitter.split_text(content)

In [ ]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=userdata.get("GOOGLE_API_KEY"))

In [ ]:
vector_index = Chroma.from_texts(text_chunks, embeddings).as_retriever(search_kwargs={"k":5})

#Question answer bot final step before fine tuning the answers

In [ ]:
template_for_prompt_response="""
Use the following pieces of context to answer the question at the end. If the question is not related to the context, politely respond that you are tuned to only answer questions that are related to the context.
dont make up your own answers. Explain to the question with maximum answer that u can give. Always greet with Thanks for asking i am happy to help you out only at the end of the answer.
Also greet them when they are saying thankyou or related to thank you.

{context}
Question: {question}
Helpful Answer:
"""
Question_answer_chain_promt_adjusting_control=PromptTemplate.from_template(template_for_prompt_response)

In [ ]:
Question_answer_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={'prompt':Question_answer_chain_promt_adjusting_control}
)

#Testing with temporary hosting

In [ ]:
import gradio as g

In [ ]:
def text_response(Question):
  question=Question
  result=Question_answer_chain({"query":question})
  return result['result']

# Create a Gradio interface
interface = g.Interface(fn=text_response, inputs="text", outputs='markdown',description='This chatbot is trained on a particular subject with the contents available related to the particular subject.',title='SUBJECT EXPERT CHATBOT TESTING PHASE-1')

# Launch the Gradio interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b463af4a0933ec2a7d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


#using lamma parse

In [ ]:
!pip install llama-parse
!pip install llama-index

In [ ]:
from google.colab import userdata
from IPython.display import display,Markdown
import textwrap

In [ ]:
path='/content/drive/MyDrive/ContentforModelFluidMechanics'

In [ ]:
# Uncomment if you are in a Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse  # pip install llama-parse
from llama_index.core import SimpleDirectoryReader  # pip install llama-index

parser = LlamaParse(
    api_key=userdata.get('LAMMA_PARSE_API_KEY'),  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown"  # "markdown" and "text" are available
)

file_extractor = {".pdf": parser}
reader = SimpleDirectoryReader(path, file_extractor=file_extractor)
documents = reader.load_data()

Started parsing the file under job_id c8e6a685-b75c-4675-af3c-996c16887909
........................................................................

In [ ]:
len(documents)

807

In [ ]:
to_markdown(documents[0].text)

AttributeError: 'list' object has no attribute 'text'

In [ ]:
# Define the file name
file_name = "fluidmechanics2500solvedproblems_extracted_file.txt"

# Create the file (if it doesn't exist) and open it in write mode
with open(file_name, "w") as file:
  pass  # Do nothing here, just creating the file

# Now, open the file in append mode to add content
with open(file_name, "a") as file:
  for i in range(len(documents)):
    file.write(documents[i].text)